In [73]:
from lxml import html
import shutil
import psycopg2 as pg
from urllib import request
import os.path

DSN = 'dbname=nvdimp user=psilotum'
conn = pg.connect(DSN)

In [2]:
getList = '''
SELECT 
    h.hast_genus_id,h.genus,t."tropicosNamecode" 
FROM 
    hast_genus h 
LEFT OUTER JOIN 
    hast_genus_w_tropicos t ON t.hast_genus_id = h.hast_genus_id
WHERE h.family_apgiv is null and "tropicosNamecode" is not null;'''
with conn:
    with conn.cursor() as curs:
        curs.execute(getList)
        gen = curs.fetchall()

In [11]:
# download html first
for i in range(0,len(gen)):
    tropicos_base_url = 'http://www.tropicos.org/Name/'
    namecode = str(gen[i][2])
    genus_name = gen[i][1]
    
    tropicos_query_url = tropicos_base_url + namecode
    if os.path.isfile('html/%s.html' % genus_name) is not True:
        sresp, shead = request.urlretrieve(tropicos_query_url)
        # cache file
        shutil.copyfile(sresp, 'html/%s.html' % genus_name)

In [151]:
for i in range(0,len(gen)):
    html_parser = html.parse('html/%s.html' % gen[i][1] ).getroot()
    higher_taxa_path = '''//ul[@style="margin-top: 0px;"]//li//a[starts-with(@id, 'ctl00_MainContentPlaceHolder_nameDetailsControl_HigherTaxonList_')]'''
    if html_parser.xpath(higher_taxa_path) == []:
        pass
    else:
        matches = html_parser.xpath(higher_taxa)
        
        if matches[len(matches)-1].text is None:
            family = ''
            family_namecode = ''
            print([i, gen[i][1]])
        else:
            family_namecode = matches[len(matches)-1].get('href').split('/')[2]
            family = matches[len(matches)-1].text.split(' ')[0].replace('!', '')
        
            UPDATE_HAST = '''UPDATE 
                    nomenclature.hast_genus 
               SET 
                    family_apgiv = '%s'
               WHERE
                    genus='%s'; ''' % (family, gen[i][1])
            with conn:
                with conn.cursor() as curs:
                    curs.execute(UPDATE_HAST)

[554, 'Heterosmilax']
[1516, 'Erythropalum']


In [159]:
# 手動處理例外的
excepGenus = ['Heterosmilax', 'Erythropalum']
for i in range(len(excepGenus)):
    html_parser = html.parse('html/%s.html' % excepGenus[i] ).getroot()
    higher_taxa_path = '''//ul[@style="margin-top: 0px;"]//li//a[starts-with(@id, 'ctl00_MainContentPlaceHolder_nameDetailsControl_HigherTaxonList_')]'''
    if html_parser.xpath(higher_taxa_path) == []:
        pass
    else:
        matches = html_parser.xpath(higher_taxa)
        family_namecode = matches[4].get('href').split('/')[2]
        family = matches[4].text.split(' ')[0].replace('!', '')
        UPDATE_HAST = '''UPDATE 
                    nomenclature.hast_genus 
               SET 
                    family_apgiv = '%s'
               WHERE
                    genus='%s'; ''' % (family, excepGenus[i])
        with conn:
            with conn.cursor() as curs:
                curs.execute(UPDATE_HAST)

In [155]:
matches[4].text

'Olacaceae R. Br.'

In [165]:
# 更新 family_id
UPDATE_HAST_FAMILY_ID = '''UPDATE hast_genus SET hast_family_id=f.hast_family_id 
    FROM 
        (SELECT * FROM hast_families) as f 
    WHERE 
        f.family=hast_genus.family_apgiv;
        
    UPDATE hast_genus SET 
        family_id_apgiv = f.family_id_apgiv, family_subid_apgiv = f.family_subid_apgiv
    FROM
        (SELECT * FROM nomenclature.apgiv_families) as f
    WHERE 
        f.family = hast_genus.family_apgiv;
        
        '''
with conn:
    with conn.cursor() as curs:
        curs.execute(UPDATE_HAST_FAMILY_ID)

In [171]:
# CREATE VIEW
CREATE_QUES_GENUS = '''
CREATE OR REPLACE VIEW nomenclature.hast_questionable_genus as 
    (SELECT 
        h.hast_genus_id, h.hast_family_id, h.family_apgiv, t.genus, h.genus_zh, 
        t."acceptedGenus", t."acceptedNamecode" 
     FROM hast_genus h 
     left outer join hast_genus_w_tropicos t 
 ON t.hast_genus_id =  h.hast_genus_id where "acceptedGenus" is not null
     or family_apgiv is null
     ORDER BY h.family_apgiv, t."acceptedGenus");
 
 CREATE OR REPLACE VIEW nomenclature.hast_checked_genus as 
    (SELECT 
        h.hast_genus_id, h.hast_family_id, h.family_apgiv, t.genus, h.genus_zh, 
        t."acceptedGenus", t."acceptedNamecode" 
     FROM hast_genus h 
     left outer join hast_genus_w_tropicos t 
 ON t.hast_genus_id =  h.hast_genus_id where "acceptedGenus" is null and 
     family_apgiv is not null order by h.family_apgiv,t.genus);
'''
with conn:
    with conn.cursor() as curs:
        curs.execute(CREATE_QUES_GENUS)